#periodic


In [172]:
import numpy as np
import networkx as nx

def cost(x, size):
    for n in range(3):
        for m in range(3):
            J = np.sum(np.exp(np.sqrt((x[:, 0] - (n * (size)/2))**2 + (x[:, 1] - (m * (size)/2))**2)))
    return J

def gradient(x, i, size, gradient_step, N):
    u = x[i, :, :]
    h = gradient_step
    u_x = np.zeros((N, 2))
    u_y = np.zeros((N, 2))
    u_x[i, 0] = h
    u_y[i, 1] = h
    grad_x = (cost(u + u_x, size) - cost(u, size)) / h
    grad_y = (cost(u + u_y, size) - cost(u, size)) / h
    grad = np.array([grad_x, grad_y])
    return grad

def consensus(a, x, N):
    for i in range(N):
        for j in range(N):
            if a[i, j] == 1:
                x[i, j, :] = x[j, j, :]
            else:
                x[i, j, :] = x[i, j, :]
    return x

def opt(x, i, step, size, gradient_step, N):
    u = x[i, :, :]
    v = u[i, :] - (step * gradient(x, i, size, gradient_step, N)) / np.linalg.norm(gradient(x, i, size, gradient_step, N))
    u[i, :] = v
    return u

def check(pos, SIZE):
    x, y = pos
    # Check if the position is out of bounds
    x = max(0, min(x, SIZE))
    y = max(0, min(y, SIZE))
    return np.array([x, y])

def nearest_feasible_position(pos, SIZE):
    # Adjust the position to the nearest feasible one within bounds
    x, y = pos
    x = max(0, min(x, SIZE))
    y = max(0, min(y, SIZE))
    return np.array([x, y])

def coverage_constraint_check(x, N, threshold=3):
    # Check if each point is covered by at least three sensors
    binary_vars = np.zeros((5, N), dtype=int)
    for i in range(5):
        distances = np.linalg.norm(x[:, :, :] - np.array([i * 100, i * 100]).reshape(1, 1, 2), axis=2)
        binary_vars[i, :] = np.sum(distances <= threshold, axis=1)

    return np.all(binary_vars >= threshold)

def threshold_distance_constraint_check(x, N, threshold=100):
    # Check if the distance between each sensor and each point is within the threshold
    distances = np.linalg.norm(x[:, :, :] - np.arange(5).reshape(5, 1, 1) * 100, axis=2)
    return np.all(distances <= threshold)

def convergence_check(previous_positions, current_positions, threshold=1e-6):
    # Check if the positions have converged based on a threshold
    return np.all(np.linalg.norm(previous_positions - current_positions, axis=-1) < threshold)


def new_net(N):
    G = nx.connected_watts_strogatz_graph(N, 4, p=2, seed=42)
    adjacency_matrix = nx.adjacency_matrix(G).todense()
    adjacency_matrix = np.array(adjacency_matrix)
    np.fill_diagonal(adjacency_matrix, 1)
    return adjacency_matrix

def exe(size, N, step, iteration, gradient_step):
    x = np.random.random(size=(N, N, 2)) * 100
    starting_positions = np.zeros((N, 2))
    finishing_positions = starting_positions.copy()

    # init network
    a = new_net(N)

    for k in range(iteration):
        print("iteration number", k)

        # consensus between connected agents
        w = consensus(a, x, N)

        # optimize each agent locally
        for j in range(N):
            u = opt(w, j, step, size, gradient_step, N)
            w[j] = u
            print("agent", j, "position", u)

        # check if all agents are in the space with size "size"
        for j in range(N):
            pos = w[j, j, :]
            pos = check(pos, size)
            w[j, j, :] = pos
            print("agent", j, "feasible position", pos)
            finishing_positions[j] = pos

        # constraint satisfaction
        for j in range(N):
            finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # coverage constraint check
        if not coverage_constraint_check(w, N):
            print("Coverage constraint violated. Adjusting positions.")
            for j in range(N):
                finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # threshold distance constraint check
        if not threshold_distance_constraint_check(w, N):
            print("Threshold distance constraint violated. Adjusting positions.")
            for j in range(N):
                finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # convergence check
        if k > 0 and convergence_check(finishing_positions, starting_positions):
            print("Converged!")
            break

        # update starting positions for the next iteration
        starting_positions = finishing_positions.copy()

    return finishing_positions

# Example usage
final_positions = exe(100, 5, 10, 200, 10)
print("Final Positions:")
print(final_positions)


iteration number 0
agent 0 position [[        nan         nan]
 [43.33160564 79.69862107]
 [66.96996457  1.68551694]
 [32.08913643 83.1214398 ]
 [29.18866323 12.66770184]]
agent 1 position [[69.00351849 88.16337418]
 [        nan         nan]
 [66.96996457  1.68551694]
 [32.08913643 83.1214398 ]
 [29.18866323 12.66770184]]
agent 2 position [[69.00351849 88.16337418]
 [43.33160564 79.69862107]
 [73.80130975  8.98844259]
 [32.08913643 83.1214398 ]
 [29.18866323 12.66770184]]
agent 3 position [[69.00351849 88.16337418]
 [43.33160564 79.69862107]
 [66.96996457  1.68551694]
 [        nan         nan]
 [29.18866323 12.66770184]]
agent 4 position [[69.00351849 88.16337418]
 [43.33160564 79.69862107]
 [66.96996457  1.68551694]
 [32.08913643 83.1214398 ]
 [36.25288622 19.74560785]]
agent 0 feasible position [0 0]
agent 1 feasible position [0 0]
agent 2 feasible position [73.80130975  8.98844259]
agent 3 feasible position [0 0]
agent 4 feasible position [36.25288622 19.74560785]
Coverage constra

C:\Users\SHAAHIN\AppData\Local\Temp\ipykernel_23868\1020537151.py:33: RuntimeWarning: invalid value encountered in divide
  v = u[i, :] - (step * gradient(x, i, size, gradient_step, N)) / np.linalg.norm(gradient(x, i, size, gradient_step, N))


agent 0 position [[98.99494937 98.99494937]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]]
agent 1 position [[91.92388155 91.92388155]
 [98.99494937 98.99494937]
 [98.99494937 98.99494937]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]]
agent 2 position [[91.92388155 91.92388155]
 [91.92388155 91.92388155]
 [91.92388155 91.92388155]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]]
agent 3 position [[91.92388155 91.92388155]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]
 [98.99494937 98.99494937]
 [98.99494937 98.99494937]]
agent 4 position [[91.92388155 91.92388155]
 [91.92388155 91.92388155]
 [98.99494937 98.99494937]
 [91.92388155 91.92388155]
 [91.92388155 91.92388155]]
agent 0 feasible position [98.99494937 98.99494937]
agent 1 feasible position [98.99494937 98.99494937]
agent 2 feasible position [91.92388155 91.92388155]
agent 3 feasible position [98.99494937 98.99494937]
agent 4 feasible position [

#Gossip

In [179]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def cost(x, size):
    for n in range(3):
        for m in range(3):
            J = np.sum(np.exp(np.sqrt((x[:, 0] - (n * size))**2 + (x[:, 1] - (m * size))**2)))
    return J

def gradient(x, i, size, gradient_step, N):
    u = x[i, :, :]
    h = gradient_step
    u_x = np.zeros((N, 2))
    u_y = np.zeros((N, 2))
    u_x[i, 0] = h
    u_y[i, 1] = h
    grad_x = (cost(u + u_x, size) - cost(u, size)) / h
    grad_y = (cost(u + u_y, size) - cost(u, size)) / h
    grad = np.array([grad_x, grad_y])
    return grad

def consensus(a, x, N):
    for i in range(N):
        for j in range(N):
            if a[i, j] == 1:
                x[i, j, :] = x[j, j, :]
            else:
                x[i, j, :] = x[i, j, :]
    return x

def opt(x, i, step, size, gradient_step, N):
    u = x[i, :, :]
    v = u[i, :] - (step * gradient(x, i, size, gradient_step, N)) / np.linalg.norm(gradient(x, i, size, gradient_step, N))
    u[i, :] = v
    return u

def check(pos, SIZE):
    x, y = pos
    # Check if the position is out of bounds
    x = max(0, min(x, SIZE))
    y = max(0, min(y, SIZE))
    return np.array([x, y])

def nearest_feasible_position(pos, SIZE):
    # Adjust each coordinate to the nearest feasible one within bounds
    x = max(0, min(pos[0], SIZE))
    y = max(0, min(pos[1], SIZE))
    return np.array([x, y])

def coverage_constraint_check(x, N, threshold=3):
    # Check if each point is covered by at least three sensors
    binary_vars = np.zeros((5, N), dtype=int)
    for i in range(5):
        distances = np.linalg.norm(x[:, :, :] - np.array([i * 100, i * 100]).reshape(1, 1, 2), axis=2)
        binary_vars[i, :] = np.sum(distances <= threshold, axis=1)

    return np.all(binary_vars >= threshold)

def threshold_distance_constraint_check(x, N, threshold=100):
    # Check if the distance between each sensor and each point is within the threshold
    distances = np.linalg.norm(x[:, :, :] - np.arange(5).reshape(5, 1, 1) * 100, axis=2)
    return np.all(distances <= threshold)

def convergence_check(previous_positions, current_positions, threshold=1e-6):
    # Check if the positions have converged based on a threshold
    return np.all(np.linalg.norm(previous_positions - current_positions, axis=-1) < threshold)

def new_net(N):
    G = nx.grid_2d_graph(int(np.sqrt(N)), int(np.sqrt(N)), periodic=True)
    adjacency_matrix = nx.adjacency_matrix(G).todense()
    return np.array(adjacency_matrix)

def gossip_algorithm(x, size, N, max_iterations=200, gradient_step=10, gossip_step=5):
    starting_positions = x.copy()
    finishing_positions = x.copy()

    # init network
    a = new_net(N)

    for k in range(max_iterations):
        print("iteration number", k)

        # gossip step
        gossip_neighbors = np.random.choice(N, size=int(N * gossip_step / 100), replace=False)
        for j in gossip_neighbors:
            neighbor = np.random.choice(N)
            x[j, :, :] = x[neighbor, :, :]

        # consensus between connected agents
        x = consensus(a, x, N)

        # optimize each agent locally
        for j in range(N):
            u = opt(x, j, gradient_step, size, gradient_step, N)
            x[j] = u
            print("agent", j, "position", u)

        # check if all agents are in the space with size "size"
        for j in range(N):
            pos = x[j, j, :]
            pos = check(pos, size)
            x[j, j, :] = pos
            print("agent", j, "feasible position", pos)
            finishing_positions[j] = pos

        # constraint satisfaction
        for j in range(N):
            finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # coverage constraint check
        if not coverage_constraint_check(x, N):
            print("Coverage constraint violated. Adjusting positions.")
            for j in range(N):
                finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # threshold distance constraint check
        if not threshold_distance_constraint_check(x, N):
            print("Threshold distance constraint violated. Adjusting positions.")
            for j in range(N):
                finishing_positions[j] = nearest_feasible_position(finishing_positions[j], size)

        # convergence check
        if k > 0 and convergence_check(finishing_positions, starting_positions):
            print("Converged!")
            break

    return finishing_positions

def plot_cost_function(x, size):
    cost_values = []
    for n in range(3):
        for m in range(3):
            cost_values.append(cost(x[:, :, :], size))
    plt.plot(cost_values, label="Cost Function")
    plt.xlabel("Iterations")
    plt.ylabel("Cost")
    plt.legend()
    plt.show()

def plot_final_positions(x, size):
    plt.scatter(x[:, :, 0], x[:, :, 1], label="Final Positions", marker='o')
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    plt.xlim(0, size)
    plt.ylim(0, size)
    plt.legend()
    plt.show()

# Example usage
final_positions_gossip = gossip_algorithm(np.random.random(size=(25, 25, 2)) * 100, 100, 25, max_iterations=200, gradient_step=10, gossip_step=5)
print("Final Positions (Gossip Algorithm):")
print(final_positions_gossip)

# Plot the cost function
plot_cost_function(final_positions_gossip, 100)

# Plot the final sensor positions
plot_final_positions(final_positions_gossip, 100)


iteration number 0
agent 0 position [[29.02525007 11.58730492]
 [60.65614081 10.6520614 ]
 [36.16852475 74.93179765]
 [ 7.19434854 32.82152591]
 [90.76066215 40.34862818]
 [25.84726104 75.41123594]
 [39.05604921 63.99546863]
 [31.29800371 52.5188294 ]
 [88.66117273 16.58773627]
 [93.83638048 30.57999302]
 [27.97018932 21.51565375]
 [39.89627131 98.40376915]
 [69.24811164  3.43804311]
 [51.82952253 73.50325203]
 [66.71222024 89.6834651 ]
 [90.94187527 77.03340883]
 [ 8.22365479 23.72286625]
 [42.17712437 34.01678961]
 [11.87247545 58.99788815]
 [52.13863189 95.34482839]
 [77.94999037 73.34926369]
 [40.53500695 15.54925834]
 [75.35363798 84.12504151]
 [10.1756813  84.17488729]
 [29.90125436  6.58069703]]
agent 1 position [[21.95648768  4.51393244]
 [67.71797547 17.73235032]
 [58.81682626 32.14774699]
 [ 4.10908003 43.00357832]
 [68.45517665  3.35392443]
 [84.86886475 80.73537539]
 [36.43802704 14.94835436]
 [66.80235554 29.30160178]
 [99.08039511  1.14537952]
 [23.28882078 11.10096669]
 

C:\Users\SHAAHIN\AppData\Local\Temp\ipykernel_23868\1608406290.py:34: RuntimeWarning: invalid value encountered in divide
  v = u[i, :] - (step * gradient(x, i, size, gradient_step, N)) / np.linalg.norm(gradient(x, i, size, gradient_step, N))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()